In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import pyodbc
import pandas as pd  # sure takes a long time
import dizzy_util as util
import numpy as np
import umap
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import cluster_umap
from collections import defaultdict

In [3]:
conn = pyodbc.connect("DRIVER={SQL Server};SERVER=VHACDWRB03;DATABASE=ORD_Singh_201911038D")

In [3]:
dataset = util.extractDataset("B00")

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_01_04_cohort,BASE TABLE
1,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_01_04_Demorgraphics,BASE TABLE
2,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_04_04_Lab,BASE TABLE
3,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_05_04_Rad,BASE TABLE
4,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_06_Med_04_RxOutpat,BASE TABLE
5,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_06_Med_05_BCMA_Dispense...,BASE TABLE
6,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_06_Med_07_NonVAMed,BASE TABLE
7,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_06_Med_08_AllMed,BASE TABLE
8,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_08_Consult,BASE TABLE
9,ORD_Singh_201911038D,Dflt,_B00_ML4TrgPos_Y201621_12_ICD,BASE TABLE


dict_keys(['cohort', 'Demorgraphics', 'Lab', 'Rad', 'RxOutpat', 'DispensedDrug', 'NonVAMed', 'AllMed', 'Consult', 'ICD', 'Vital', 'HF', 'Note'])


In [19]:
abdominal_df = pd.read_csv(open("Viral_data/ML4_HighRisk_AbdominalPain/labels-AbdominalPain.csv", 'r', errors="ignore"))
dizziness_df = pd.read_csv(open("Viral_data/ML4_HighRisk_Dizziness/labels-Dizziness.csv", 'r', errors="ignore"))

In [24]:
abdominal_df.columns

Index(['AutoID', 'StudyID', 'ReviewerID', 'ReviewDate', 'PtSSN', 'Facility',
       'Age', 'DOBPt', 'Gender', 'Race', 'TriggerIncAdmissionER',
       'TriggerIncAbdominalPainER', 'TriggerIncFeverHypoER',
       'TriggerIncInPtAdm', 'TriggerExclPalliativeVisit',
       'TriggerExclSameDayAdmDis', 'TriggerExclTerminalIllness',
       'TriggerExclExternalCause', 'TriggerExclPscyAdmission',
       'TriggerExclAge', 'InPtAdmDt', 'InPtAdmType', 'InPtDx',
       'InPtAdditionalDx', 'DischargeDate', 'DischargeDx',
       'DischargeAdditionalDx', 'IndexERDate', 'NoERVisit3Months',
       'NoHospitalization1year', 'ProviderType', 'ChiefComlERText',
       'AbdPainOnset', 'AbdPainDuration', 'AbdPainLocation',
       'AbdPainRadiation', 'AbdPainRadiatonText', 'AbdPainQuality',
       'AbdPainSeverity', 'AbdPainProvokingFact', 'AbdPainPalliatingFact',
       'AbdPainAssociatedSymptoms', 'PMHAbdSurgery', 'PMHAbdSurgeryText',
       'PMHMentalHealth', 'PMHMentalHealthText', 'PMHSubstanceAbuse',
     

In [26]:
abdominal_df.dropna(subset=["DxError", "PtSSN"], inplace=True)
abdominal_df.DxError

0     5.0
1     1.0
2     3.0
3     2.0
5     2.0
     ... 
95    5.0
96    2.0
97    1.0
98    3.0
99    2.0
Name: DxError, Length: 87, dtype: float64

In [11]:
icd_df = dataset["ICD"]

In [ ]:
display(icd_df)

# Procedural

In [44]:
lab_df = pd.read_sql(sql="select * from Dflt._B00_ML4TrgPos_Y201621_04_04_Lab t", con=conn)

In [ ]:
display(lab_df)

In [ ]:
lab_df.loc[lab_df['LabChemTestName'].str.contains("neur", case=False)]

In [48]:
lab_df.loc[lab_df['LabChemTestName'].str.contains("MRI", case=False)]

,PatientSSN,LabChemSpecimenDateTime,LabChemResultValue,LabChemResultNumericValue,Abnormal,LabChemTestName,RequestLocation,LOINC,LabChemSID,Sta3n,LabChemTestSID,PatientSID


In [52]:
lab_df.loc[lab_df['LOINC'].str.contains("6406", case=False)]
# LOINC of MRI = LP6406-5?

,PatientSSN,LabChemSpecimenDateTime,LabChemResultValue,LabChemResultNumericValue,Abnormal,LabChemTestName,RequestLocation,LOINC,LabChemSID,Sta3n,LabChemTestSID,PatientSID


In [51]:
lab_df.loc[lab_df['LabChemTestName'].str.contains("imaging", case=False)]

,PatientSSN,LabChemSpecimenDateTime,LabChemResultValue,LabChemResultNumericValue,Abnormal,LabChemTestName,RequestLocation,LOINC,LabChemSID,Sta3n,LabChemTestSID,PatientSID


In [55]:
lab_df.loc[lab_df['LabChemTestName'].str.contains("img", case=False)]

,PatientSSN,LabChemSpecimenDateTime,LabChemResultValue,LabChemResultNumericValue,Abnormal,LabChemTestName,RequestLocation,LOINC,LabChemSID,Sta3n,LabChemTestSID,PatientSID


In [53]:
lab_df.loc[lab_df['LabChemTestName'].str.contains("radiology", case=False)]

,PatientSSN,LabChemSpecimenDateTime,LabChemResultValue,LabChemResultNumericValue,Abnormal,LabChemTestName,RequestLocation,LOINC,LabChemSID,Sta3n,LabChemTestSID,PatientSID


In [16]:
consult_df = pd.read_sql(sql="select * from Dflt._B00_ML4TrgPos_Y201621_08_Consult t", con=conn)

In [ ]:
display(consult_df)

In [ ]:
neur_service_df = consult_df.loc[consult_df['ToRequestServiceName'].str.contains("neur", case=False)]
display(neur_service_df)

In [22]:
neur_service_df.ConStopCode.value_counts()

315.0    1153
406.0     326
697.0     214
325.0     186
669.0     161
         ... 
403.0       1
512.0       1
213.0       1
582.0       1
690.0       1
Name: ConStopCode, Length: 67, dtype: int64

In [21]:
neur_service_df.ToRequestServiceName.value_counts()

NEUROLOGY OUTPT                                          174
NEUROLOGY OUTPATIENT                                     171
NEUROLOGY INPATIENT                                      122
NEUROLOGY OUTPT CONSULT                                   96
NEUROLOGY INPT                                            87
                                                        ... 
NEUROSURGERY INPATIENT/BT                                  1
NEUROLOGY - CP OUTPT                                       1
IFC NEUROSURGERY CRANIAL/SPINAL TUMOR/SPINAL MASS GLA      1
IFC NEUROSURGERY CLINIC (SAN FRAN)                         1
NEUROSURGERY E-CONSULT (POR) IFC                           1
Name: ToRequestServiceName, Length: 497, dtype: int64

In [12]:
consult_df[consult_df.ConStopCodeName.str.contains("neurology", case=False, na=False)].ConStopCode.unique()

array([315., 325.])

In [ ]:
consult_df[consult_df.ConStopCode == 325.0]

In [15]:
temp1 = consult_df[consult_df['ToRequestServiceName'].str.contains("neur", case=False)]["ToRequestServiceName"].unique()
print(len(temp1))
print(temp1)

497
['NEUROLOGY OUTPATIENT' 'Neurology Inpt'
 'Neuropsychological Assessment Outpt' 'NEUROLOGY GENERAL OUTPT (SAC)'
 'NEUROLOGY HEADACHE OUTPT' 'NEUROLOGY ED OUTPT CONSULT' 'NEUROLOGY INPT'
 'NEUROLOGY OUTPT' 'NEUROSURGERY IN/OUT PT (OLD) OUTPT'
 'NEUROLOGY ECONSULT OUTPT'
 'P NEUROSURGERY-ECONSULT REVIEW IFC (PORTLAND)'
 'E-CONSULT NEUROSURGERY OUTPT' 'E-CONSULT NEUROLOGY OUTPT'
 'NEUROPSYCHOLOGY OUTPATIENT' 'Neurology E-Consult Outpt'
 'Formulary Exception - Neurology Outpt PADR'
 'COGNITIVE DISORDERS - NEUROPSYCHOLOGICAL TESTING OUTPT'
 'NEUROSURGERY CLINIC OUTPT' 'NEUROSURGERY INPT'
 'NEUROLOGY INPT (GENERAL)'
 'CO-BH NEUROPSYCHOLOGICAL TESTING OUTPATIENT-589A4'
 'IC/NEUROLOGY OUTPT-636A8' 'EMG/NCV NEUROLOGY OUTPT OR INPAT'
 'NEURODIAGNOSTICS OUTPATIENT' 'NEUROLOGY OUTPATIENT JC'
 'NEUROPSYCHOLOGICAL OUTPT TESTING JB' 'NEUROLOGY (INPATIENT) CONSULT'
 'NEUROLOGY EMG (CCD*) OUTPT' 'NEUROLOGY (CCD*) - OUTPT' 'NEUROLOGY-OUTPT'
 'NEUROLOGY INPATIENT' 'NEUROSURGERY OUTPT' 'zzFM NEUROLOGY

In [49]:
temp1 = dataset["Consult"].loc[dataset["Consult"]['ToRequestServiceName'].str.contains("neur", case=False)].ConStopCodeName.unique()
display(temp1)

array(['NEUROLOGY', 'PSYCHOLOGICAL TESTING',
       'TBI (TRAUMATIC BRAIN INJURY)', 'NEUROSURGERY', 'CHART CONSULT',
       'CLINICAL PHARMACY', 'PSYCHOLOGY', 'SLEEP MEDICINE', 'EEG',
       'EMG - ELECTROMYOGRAM', 'DEMENTIA CLINIC', 'PM&RS PHYSICIAN',
       'MENTAL HEALTH CLINIC - IND', 'NURSE PRACTITIONER',
       'EPILEPSY ECOE', 'POLYTRAUMA/TBI IND', 'ALS CENTER',
       'COMMUNITY CARE CONSULT', 'TELEPHONE/NEUROLOGY',
       'SPINAL CORD INJURY', 'EVOKED POTENTIAL', 'PAIN CLINIC',
       'ADMIN PAT ACTIVTIES (MASNONCT)', 'POST SURG RTINE AFTRCARE 2ND',
       'PM&RS AMP CLINIC', 'TELEPHONE/SURGERY', 'TELEPHONE/MEDICINE',
       'TELEPHONE MH', 'EMERGENCY DEPT', 'MH INTGRTD CARE IND',
       'ORTHO/JOINT SURG', 'MH INTERVNTION BIOMED CARE IND',
       'RT CLIN VD TH PRV SITE(DIFSTA)', 'SPINAL SURG', None,
       'RT CLIN VID CARE HOME', 'MULTIPLE SCLEROSIS (MS)',
       'PADRECC PARKINSONS', 'PSYCHIATRY', 'PHYSICAL THERAPY',
       'MENTAL HEALTH CONSULTATION', 'FELLOW/RESIDENT',


In [ ]:
temp2 = consult_df.loc[consult_df['ConStopCode'] == 315.0]
display(temp2)
print(len(temp2.ToRequestServiceName.unique()))
display(temp2[temp2.requestType == "P"])

In [ ]:
rad_df = pd.read_sql(sql="select * from Dflt._B00_ML4TrgPos_Y201621_05_04_Rad t", con=conn)
display(rad_df)

In [ ]:
rad_df[rad_df.CPTCode.str.contains("70", na=False)]

In [6]:
mri1 = rad_df.loc[rad_df['CPTName'].str.contains(" mri", case=False, na=False)]
mri2 = rad_df.loc[rad_df['CPTName'].str.contains(" mri ", case=False, na=False)]
mri3 = rad_df.loc[rad_df['CPTName'].str.contains("mri ", case=False, na=False)]

In [ ]:
mri_df = pd.concat([mri1, mri2, mri3])
mri_df = mri_df.reset_index(drop=True)
display(mri_df)

In [13]:
mri_brain_df = util.filterDFByCodeSet(rad_df, "CPTCode", util.MRI_CPT_CODES).copy()

In [15]:
util.displayAll(mri_brain_df.RadiologyDiagnosticCode.value_counts())

NO ALERT REQUIRED                                             155
SIGNIFICANT ABNORMALITY, ATTN NEEDED                          109
MINOR ABNORMALITY                                              50
ABNORMALITY, ATTN. NEEDED                                      43
CRITICAL ABNORMALITY                                           20
Abnormality Follow-up Needed                                   20
Minor Abnormality or Abn Prev Identified                       17
ABNORMAL                                                       17
NORMAL                                                         14
MAJOR ABNORMALITY, PHYSICIAN AWARE                             11
ABNORMAL, ATTENTION NEEDED                                      9
ABNORMALITY, CORRELATE CLINICALLY                               5
MAJOR ABNORMALITY, NO ATTN. NEEDED                              5
ABNORMAL, ATTENTION NEEDED, E-alert Sent                        4
Abnormality Follow-Up Needed                                    4
Outside Re

In [24]:
mri_brain_df.RadiologyDiagnosticCode.value_counts().to_csv("mri_brain.csv")

In [27]:
rad_diag_df = util.filterDFByCodeSet(rad_df, "CPTCode", util.MRI_CPT_CODES + util.CT_CPT_CODES).copy()
display(rad_diag_df.RadiologyDiagnosticCode.value_counts())
rad_diag_df.RadiologyDiagnosticCode.value_counts().to_csv("mri_ct_brain.csv")

NO ALERT REQUIRED                           944
SIGNIFICANT ABNORMALITY, ATTN NEEDED        314
MINOR ABNORMALITY                           176
ABNORMALITY, ATTN. NEEDED                    93
Minor Abnormality or Abn Prev Identified     69
                                           ... 
NO ACUTE FINDINGS                             1
PHYSICIAN AWARE OF RESULT                     1
Outside Imaging Report                        1
Stroke                                        1
Significant Abnormality Attention Needed      1
Name: RadiologyDiagnosticCode, Length: 81, dtype: int64

In [20]:
mri_df[mri_df["CPTName"].str.contains("head", case=False, na=False)]

,PatientSSN,ExamDateTime,CPTCode,CPTName,RadiologyDiagnosticCode,RadiologyExamSID,Sta3n,PatientSID,RadiologyProcedureSID


In [ ]:
mri_brain_df = mri_df[mri_df["CPTName"].str.contains("brain", case=False, na=False)]
display(mri_brain_df)
display(mri_brain_df.CPTCode.unique())
display(mri_brain_df.CPTName.unique())

In [ ]:
ct1 = rad_df.loc[rad_df['CPTName'].str.contains(" ct", case=False, na=False)]
ct2 = rad_df.loc[rad_df['CPTName'].str.contains(" ct ", case=False, na=False)]
ct3 = rad_df.loc[rad_df['CPTName'].str.contains("ct ", case=False, na=False)]
ct_df = pd.concat([ct1, ct2, ct3])
ct_df = ct_df.reset_index(drop=True)
display(ct_df)

In [ ]:
ct_brain_df = ct_df[ct_df["CPTName"].str.contains("brain", case=False, na=False)]
display(ct_brain_df)
display(ct_brain_df.CPTCode.unique())
display(ct_brain_df.CPTName.unique())

In [ ]:
ct_head_df = ct_df[ct_df["CPTName"].str.contains("head", case=False, na=False)]
display(ct_head_df)
display(ct_head_df.CPTCode.unique())
display(ct_head_df.CPTName.unique())

In [ ]:
rad_df.loc[rad_df['CPTName'].str.contains("magn", case=False, na=False)]

In [ ]:
rad_df.loc[rad_df['CPTName'].str.contains(" ct ", case=False, na=False)]

In [ ]:
rad_df.loc[rad_df['CPTName'].str.contains(" ct", case=False, na=False)]

In [ ]:
dataset["Rad"].loc[dataset["Rad"]['CPTName'].str.contains("ct ", case=False, na=False)]

In [ ]:
dataset["Rad"].loc[dataset["Rad"]['CPTName'].str.contains("cat", case=False, na=False)]

In [81]:
rad_df.loc[rad_df['CPTName'].str.contains("tomography", case=False, na=False)]

,PatientSSN,ExamDateTime,CPTCode,CPTName,RadiologyDiagnosticCode,RadiologyExamSID,Sta3n,PatientSID,RadiologyProcedureSID


In [ ]:
# what are these...
rad_df.loc[rad_df['CPTName'].str.contains("akjsdhfadkfabj", case=False, na=True)]